## To Do:
- Deal with records that have RDATE before ADATE
- (?) (Probably doesn't make sense) Do something with this information (from metadata text file):
                            LL mailings had labels only
                            WL mailings had labels only
                            CC mailings are calendars with stickers but do
                               not have labels
                            FS mailings are blank cards that fold into
                               thirds with labels
                            NK mailings are blank cards with labels
                            SK mailings are blank cards with labels
                            TK mailings have thank you printed on the
                               outside with labels
                            GK mailings are general greeting cards (an
                               assortment of birthday, sympathy, blank, & get
                               well) with labels
                            XK mailings are Christmas cards with labels
                            X1 mailings have labels and a notepad
                            G1 mailings have labels and a notepad
- Use the variable that is most correlated with 'Age' ('Age' is yet to be created from 'DOB') to fill in the missing values of 'Age' (using a linear model for example)
- As a final check for outlier detection, use DBSCAN to see if all outliers were excluded
- Henrique's notes:
    - I had one note in my notebook we should turn ODATE into number of months for RFA matters maybe and DOB to days
        - -> <span style="color:red">Turned all date features into days relative to ADATE_2</span>
    - Another one saying that NOEXCH could be remove. Check it out a see if you agree
        - <span style="color:red">This one doesn't seem too bad to me so far actually</span>
- Decide what to do with 'HOMEOWNR' (Home Owner Flag, H = Home owner, U = Unknown)
    - pd.unique(donors.HOMEOWNR) ---> array([nan, 'H', 'U'], dtype=object)
    - Could be incuded as: "Home owner" = 1, "No home owner" = 0, "Unknown" = nan
    - But does it make sense to use binary variables for clustering?
- What was done in the "clustering" lab: Use DBSCAN to remove outliers. Do the clustering on the inliers. Then in the end add each outlier to the closest cluster.

## <span style="color:red">Insights from the Q&A on Thursday, 10. Dec. 2020:</span>

<span style="color:red">**See text file "Notes Q&A 10. Dec. 2020" in folder "PDFs and notes"**</span>

## Import libraries

In [1]:
import pandas as pd
import os
import numpy as np
import datetime
import dateutil.relativedelta
from datetime import date
import math
import seaborn as sns
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
from math import ceil

%matplotlib inline

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Seeting seaborn style
sns.set()

In [2]:
# Turn off warnings
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

## Load data

In [3]:
# Load data
donors = pd.read_csv(os.path.join('Data and metadata', 'donors.csv'), sep=',', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'Data and metadata/donors.csv'

In [ ]:
donors

In [ ]:
# Save the original features in a list
features_orig = list(donors)

## Code for checking the types of all elements of a column

In [ ]:
feature = 'ADATE_2'
type_items_to_retieve = float
type_set = set()
list_ = []
for item in donors[feature]:
    type_ = type(item)
    type_set.add(type_)
    if type_ == type_items_to_retieve:
        list_.append(item)
print('Types present in this feature:')
print(type_set)
print('Items that have type', type_items_to_retieve, ':')
print(pd.unique(list_))

## Separate metric features from non-metric features

In [ ]:
# Look at top of remaining dataset
donors.head(5)

In [ ]:
## Obtain information of the type of a certain feature
feature = 'HOMEOWNR'

# Print its type
print('dtype of feature', feature, ':', donors.dtypes[feature])

# Take a closer look at the first non nan element of it
first_non_na_element = donors[feature][~donors[feature].isna()].iloc[0]
print('First non nan element of this feature:', first_non_na_element)
print('Class of the first element of this feature:', type(first_non_na_element))

# View the unique values of this feature
# print('Sorted unique values of feature', feature, ':', np.sort(pd.unique(donors[feature])))
print('Unique values of feature', feature, ':', pd.unique(donors[feature]))

In [ ]:
# Create a list with the metric features
# Including dates and other variables that might still be Strings at this point
# Will use 'to_num' on all metric features in the next step
metric_features = ['ODATEDW',
 'DOB',
 'NUMCHLD',
 'INCOME',
 'WEALTH1',
 'HIT',
                   # Above: Information about the donor
                   # Below: About the number of times the donor has responded to other types of mail order offers
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
                   # Below: Data from third party regarding the household or neighborhood (According to David Silva) 
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',         
 'SOLP3',
 'SOLIH',
 'WEALTH2',
                   # Below: About donor's neighbourhood
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',              
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',           
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU1',
 'HU2',
 'HU3',
 'HU4',
 'HU5',                  
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC2',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HVP1',
 'HVP2',
 'HVP3',
 'HVP4',
 'HVP5',
 'HVP6',                  
 'HUR1',
 'HUR2',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA2',
 'HUPA3',
 'HUPA4',
 'HUPA5',
 'HUPA6',
 'HUPA7',
 'RP1',
 'RP2',
 'RP3',
 'RP4',                 
 'IC1',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC6',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC11',
 'IC12',
 'IC13',
 'IC14',
 'IC15',
 'IC16',
 'IC17',
 'IC18',
 'IC19',
 'IC20',
 'IC21',
 'IC22',
 'IC23',           
 'HHAS1',
 'HHAS2',
 'HHAS3',
 'HHAS4',
 'MC1',
 'MC2',
 'MC3',
 'TPE1',
 'TPE2',
 'TPE3',
 'TPE4',
 'TPE5',
 'TPE6',
 'TPE7',
 'TPE8',
 'TPE9',
 'PEC1',
 'PEC2',
 'TPE10',
 'TPE11',
 'TPE12',
 'TPE13',
 'LFC1',
 'LFC2',
 'LFC3',
 'LFC4',
 'LFC5',
 'LFC6',
 'LFC7',
 'LFC8',
 'LFC9',
 'LFC10',
 'OCC1',
 'OCC2',
 'OCC3',
 'OCC4',
 'OCC5',
 'OCC6',
 'OCC7',
 'OCC8',
 'OCC9',
 'OCC10',
 'OCC11',
 'OCC12',
 'OCC13',
 'EIC1',
 'EIC2',
 'EIC3',
 'EIC4',
 'EIC5',
 'EIC6',
 'EIC7',
 'EIC8',
 'EIC9',
 'EIC10',
 'EIC11',
 'EIC12',
 'EIC13',
 'EIC14',
 'EIC15',
 'EIC16',
 'OEDC1',
 'OEDC2',
 'OEDC3',
 'OEDC4',
 'OEDC5',
 'OEDC6',
 'OEDC7',
 'EC1',
 'EC2',
 'EC3',
 'EC4',
 'EC5',
 'EC6',
 'EC7',
 'EC8',
 'SEC1',
 'SEC2',
 'SEC3',
 'SEC4',
 'SEC5',
 'AFC1',
 'AFC2',
 'AFC3',
 'AFC4',
 'AFC5',
 'AFC6',
 'VC1',
 'VC2',
 'VC3',
 'VC4',
 'ANC1',
 'ANC2',
 'ANC3',
 'ANC4',
 'ANC5',
 'ANC6',
 'ANC7',
 'ANC8',
 'ANC9',
 'ANC10',
 'ANC11',
 'ANC12',
 'ANC13',
 'ANC14',
 'ANC15',
 'POBC1',
 'POBC2',
 'LSC1',
 'LSC2',
 'LSC3',
 'LSC4',
 'VOC1',
 'VOC2',
 'VOC3',
 'HC1',
 'HC2',
 'HC3',
 'HC4',
 'HC5',
 'HC6',
 'HC7',
 'HC8',
 'HC9',
 'HC10',
 'HC11',
 'HC12',
 'HC13',
 'HC14',
 'HC15',
 'HC16',
 'HC17',
 'HC18',
 'HC19',
 'HC20',
 'HC21',
 'MHUC1',
 'MHUC2',
 'AC1',
 'AC2',
                   # Above: About donor's neighbourhood      
                   # Below: Date promotion X was mailed
 'ADATE_2',
 'ADATE_3',
 'ADATE_4',
 'ADATE_5',
 'ADATE_6',
 'ADATE_7',
 'ADATE_8',
 'ADATE_9',
 'ADATE_10',
 'ADATE_11',
 'ADATE_12',
 'ADATE_13',
 'ADATE_14',
 'ADATE_15',
 'ADATE_16',
 'ADATE_17',
 'ADATE_18',
 'ADATE_19',
 'ADATE_20',
 'ADATE_21',
 'ADATE_22',
 'ADATE_23',
 'ADATE_24',
                   # Below: Information about how many promotions donor has received
 'CARDPROM',
 'MAXADATE',
 'NUMPROM',
 'CARDPM12',
 'NUMPRM12',
                   # Below: Date the donation was received
 'RDATE_3',
 'RDATE_4',
 'RDATE_5',
 'RDATE_6',
 'RDATE_7',
 'RDATE_8',
 'RDATE_9',
 'RDATE_10',
 'RDATE_11',
 'RDATE_12',
 'RDATE_13',
 'RDATE_14',
 'RDATE_15',
 'RDATE_16',
 'RDATE_17',
 'RDATE_18',
 'RDATE_19',
 'RDATE_20',
 'RDATE_21',
 'RDATE_22',
 'RDATE_23',
 'RDATE_24',
                   # Below: Dollar amount of the donation
 'RAMNT_3',
 'RAMNT_4',
 'RAMNT_5',
 'RAMNT_6',
 'RAMNT_7',
 'RAMNT_8',
 'RAMNT_9',
 'RAMNT_10',
 'RAMNT_11',
 'RAMNT_12',
 'RAMNT_13',
 'RAMNT_14',
 'RAMNT_15',
 'RAMNT_16',
 'RAMNT_17',
 'RAMNT_18',
 'RAMNT_19',
 'RAMNT_20',
 'RAMNT_21',
 'RAMNT_22',
 'RAMNT_23',
 'RAMNT_24',
                   # Below: Summary variables for this donor
 'RAMNTALL',
 'NGIFTALL',
 'CARDGIFT',
 'MINRAMNT',
 'MINRDATE',
 'MAXRAMNT',
 'MAXRDATE',
 'LASTGIFT',
 'LASTDATE',
 'FISTDATE',
 'NEXTDATE',
 'TIMELAG',
 'AVGGIFT']

print('Number of metric features:', len(metric_features))

# Save this oroginal metric features list
metric_features_orig = metric_features.copy()

In [ ]:
# Create a list with the non metrics features by excluding the metric ones
non_metric_features = donors.columns.drop(metric_features).to_list()

print('Number of non-metric features:', len(non_metric_features))

# Save this oroginal metric features list
non_metric_features_orig = non_metric_features.copy()

## Treat features where " " (space) carries a meaning

**MAILCODE**

In [ ]:
feature = 'MAILCODE'
pd.unique(donors[feature])

In [ ]:
donors[feature].replace(" ", "Address is OK", inplace=True)
donors[feature].replace("B", "Bad Address", inplace=True)
pd.unique(donors[feature])

**NOEXCH**

In [ ]:
feature = 'NOEXCH'
pd.unique(donors[feature])

In [ ]:
donors[feature].replace(" ", "can be exchanged", inplace=True)
donors[feature].replace("X", "do not exchange", inplace=True)
# Assumption: 1 = do not exchange, 0 = can be exchanged (Makes sense because of the variable name)
donors[feature].replace('0', "can be exchanged", inplace=True)
donors[feature].replace('1', "do not exchange", inplace=True)
donors[feature].replace(0, "can be exchanged", inplace=True)
donors[feature].replace(1, "do not exchange", inplace=True)
pd.unique(donors[feature])

**RECINHSE**

In [ ]:
feature = 'RECINHSE'
pd.unique(donors[feature])

In [ ]:
donors[feature].replace(" ", "Not an In House Record", inplace=True)
donors[feature].replace("X", "Donor has given to PVA's In House program", inplace=True)
pd.unique(donors[feature])

**RECP3**

In [ ]:
feature = 'RECP3'
pd.unique(donors[feature])

In [ ]:
donors[feature].replace(" ", "Not a P3 Record", inplace=True)
donors[feature].replace("X", "Donor has given to PVA's P3 program", inplace=True)
pd.unique(donors[feature])

**RECPGVG**

In [ ]:
feature = 'RECPGVG'
pd.unique(donors[feature])

In [ ]:
donors[feature].replace(" ", "Not a Planned Giving Record", inplace=True)
donors[feature].replace("X", "Planned Giving Record", inplace=True)
pd.unique(donors[feature])

**RECSWEEP**

In [ ]:
feature = 'RECSWEEP'
pd.unique(donors[feature])

In [ ]:
donors[feature].replace(" ", "Not a Sweepstakes Record", inplace=True)
donors[feature].replace("X", "Sweepstakes Record", inplace=True)
pd.unique(donors[feature])

**MAJOR**

In [ ]:
feature = 'MAJOR'
pd.unique(donors[feature])

In [ ]:
donors[feature].replace(" ", "Not a Major Donor", inplace=True)
donors[feature].replace("X", "Major Donor", inplace=True)
pd.unique(donors[feature])

## Asses existance of nan values and duplictes and deal with empty strings

In [ ]:
# See how many duplicated observations exist
donors.duplicated().sum()

In [ ]:
# List the variables that have missing values and their missing value counts
missing_value_counts = donors.isna().sum()[donors.isna().sum()!=0]
missing_value_counts

In [ ]:
# Get the total number of nan values
donors.isna().sum()[donors.isna().sum()!=0].sum()

In [ ]:
# Replace " " by nans
donors.replace(" ", np.nan, inplace=True)

In [ ]:
# Get the total number of nan values once more
donors.isna().sum()[donors.isna().sum()!=0].sum()

<span style="color:red">**About 3 million " " (spaces) were converted to nan.**</span>

In [ ]:
# Replace "" by nans
donors.replace("", np.nan, inplace=True)

In [ ]:
# Get the total number of nan values once more
donors.isna().sum()[donors.isna().sum()!=0].sum()

<span style="color:red">**No additional nans added in this step.**</span>

## Assess the percentage of missing values per feature and drop the features that have more than 40% mssing values

In [ ]:
# Create an empty list
nan_percentage_list = []

# Loop over the list of features and compute their percentage of nan values
for feature in list(donors):
    nan_percentage = len(donors[feature][donors[feature].isna()]) / len(donors[feature]) * 100
    nan_percentage_list.append(nan_percentage)
    
# Create a look-up table
nan_percentage_series = pd.Series(data=nan_percentage_list, index=list(donors))
nan_percentage_series

In [ ]:
# Get a list of the features that have a higher amount of missing values
lower_nan_percentage = nan_percentage_series[nan_percentage_series<=40].index.tolist()

# Get a list of the features that have a lower amount of missing values
higher_nan_percentage = nan_percentage_series[nan_percentage_series>40].index.tolist()

# For report: Save the names of the features with the lower nan percentage in the variable 
# "features_dropped_due_to_nans"
features_dropped_due_to_nans = higher_nan_percentage

In [ ]:
donors_1 = donors[lower_nan_percentage]
donors_1

In [ ]:
# Accept the changes
donors = donors_1

In [ ]:
def sort_features(original_order, shuffled):
    ordered = []
    for feature in original_order:
        if feature in shuffled:
            ordered.append(feature)
    return ordered

In [ ]:
# Update 'metric_features' list

# Print the number of metric features before the removal
print('Number of metric features before removal:', len(metric_features))

# Get the features that are metric features and still exist in our dataset
metric_features = list(set(metric_features).intersection(set(donors)))

# And sort them according to our original order
metric_features = sort_features(metric_features_orig, metric_features)

# Print the number of metric features after the removal
print('Number of metric features afer removal:', len(metric_features))

In [ ]:
# Update 'non_metric_features' list

# Print the number of metric features before the removal
print('Number of non-metric features before removal:', len(non_metric_features))

# Get the features that are metric features and still exist in our dataset
non_metric_features = list(set(non_metric_features).intersection(set(donors)))

# And sort them according to our original order
non_metric_features = sort_features(non_metric_features_orig, non_metric_features)

# Print the number of metric features after the removal
print('Number of non-metric features afer removal:', len(non_metric_features))

In [ ]:
# Get the total number of nan values once more
donors.isna().sum()[donors.isna().sum()!=0].sum()

<span style="color:red">**Before: About 8 million nans. Now: Less than 1 million nans.**</span>

## Transform columns containing dates to datetime format

In [ ]:
def series_string_to_date_if_not_nan(x):
    """This function checks if something is a string, and if so, converts it into a datetime object"""
    # If it's not a nan, but a string
    if type(x)==str:
        x = datetime.datetime.strptime(x, '%Y-%m-%d').date()
    # if it's a nan, change to NaT
    # else:
    #     x = pd.NaT
    return x

In [ ]:
# Define a function for converting series containing strings to series containing datetime objects
def series_string_to_date(series):
    """This function turns a pandas series that consists of String values into a pandas series containing 
    datetime objects"""
    series_datetime = series.map(lambda x: series_string_to_date_if_not_nan(x))
    return series_datetime

In [ ]:
def replace_string_col_by_date_col(df, colname):
    """This function uses 'series_string_to_date' for replacing a string column by a datetime object column"""
    df[colname] = series_string_to_date(df[colname])

In [ ]:
# Code for checking data types
if False:
    labels_types = pd.DataFrame({'Labels':list(donors), 'Data types':list(donors.dtypes)})
    labels_types[0:10]

In [ ]:
# Define the labels who's columns should be changed to datetime
date_features = [
    'ODATEDW',
    'DOB',
    'ADATE_2',
    'ADATE_3',
    'ADATE_4',
    'ADATE_5',
    'ADATE_6',
    'ADATE_7',
    'ADATE_8',
    'ADATE_9',
    'ADATE_10',
    'ADATE_11',
    'ADATE_12',
    'ADATE_13',
    'ADATE_14',
    'ADATE_15',
    'ADATE_16',
    'ADATE_17',
    'ADATE_18',
    'ADATE_19',
    'ADATE_20',
    'ADATE_21',
    'ADATE_22',
    'ADATE_23',
    'ADATE_24',
    'MAXADATE',
    'RDATE_3',
    'RDATE_4',
    'RDATE_5',
    'RDATE_6',
    'RDATE_7',
    'RDATE_8',
    'RDATE_9',
    'RDATE_10',
    'RDATE_11',
    'RDATE_12',
    'RDATE_13',
    'RDATE_14',
    'RDATE_15',
    'RDATE_16',
    'RDATE_17',
    'RDATE_18',
    'RDATE_19',
    'RDATE_20',
    'RDATE_21',
    'RDATE_22',
    'RDATE_23',
    'RDATE_24',
    'MINRDATE',
    'MAXRDATE',
    'LASTDATE',
    'FISTDATE',
    'NEXTDATE',
]

# Save this original list of date features
date_features_orig = date_features.copy()

# Get the features that are date features and still exist in our dataset
date_features = list(set(date_features).intersection(set(donors)))

# And sort them according to our original order
date_features = sort_features(date_features_orig, date_features)

In [ ]:
# Code for checking the existing datatypes in a given columns
# This needs some work. Doesn't always work properly
if False:
    feature = 'NUMCHLD'
    unique_types = set()
    str_indices = []
    float_indices = []
    other_indices = []
    for i in range(len(donors[feature])):
        if type(donors[feature][i])==str:
                str_indices.append(i)
        if type(donors[feature][i])==float:
                float_indices.append(i)
        else:
                other_indices.append(i)
        unique_types.add(type(donors[feature][i]))
    nans = donors[feature][float_indices]
    strings = donors[feature][str_indices]
    print('Strings:', strings)
    print('Floats (can be NaNs):', nans)
    print('Contains the following data types:', unique_types)

In [ ]:
# Use the functions defined above to change all non-nan values of the columns in 'date_features' datetime objects
for label_to_change in date_features:
    replace_string_col_by_date_col(donors, label_to_change)

In [ ]:
# Check that it worked properly
for i in range(len(date_features)):
    print(type(donors[date_features[i]][0]))

In [ ]:
# Check if time calculations work properly with these columns
now = date.today()
now - donors['DOB']
# Seems to work well.

## Create pandas-profiling report

In [ ]:
now_string = str(datetime.datetime.now())

profile = ProfileReport(
    donors, 
    title='Donors Data',
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
    minimal = False
)

In [ ]:
folder_path = os.path.join('pandas-profiling', now_string)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [ ]:
if False:
    profile.to_file(os.path.join(folder_path, "pandas_profiling.html"))

## Data cleaning

In [ ]:
# Adapted from "lab04_data_visualization" (Data Mining)
def single_hist(feature, savefig, *n_of_bins):
    """This function takes feature name and produces a visualisation of the histogram of the respective feature"""
    
    plt.figure(figsize=(15,10))
    
    # Single Metric Variable Histogram
    if len(n_of_bins)==0:
        print('Number of bins: Automatic')
        plt.hist(donors[feature])
    else:
        print('Number of bins:', n_of_bins[0])
        plt.hist(donors[feature], bins = n_of_bins[0])
    plt.title(feature, y=-0.1)
    
    if savefig:
        # Save figure and include time stamp in filename
        now_string = str(datetime.datetime.now())[0:19].replace(':', '-').replace(' ', '_')
        folder_path = os.path.join('figures')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        filename = now_string + '_' + feature + '_single_histogram.png'
        plt.savefig(os.path.join(folder_path, filename), dpi=200)

    plt.show()

In [ ]:
# Adapted from "lab04_data_visualization" (Data Mining)

def many_hists(features, savefig, *n_of_bins):
    """This function takes a list of features and optionally a number of bins and creates histograms of these
    features."""
    
    # All Numeric Variables' Histograms in one figure
    sns.set()

    # Prepare figure. Create individual axes where each histogram will be placed
    fig, axes = plt.subplots(2, ceil(len(features) / 2), figsize=(20, 11))

    # Plot data
    # Iterate across axes objects and associate each histogram (hint: use the ax.hist() instead of plt.hist()):
    for ax, feat in zip(axes.flatten(), features): # Notice the zip() function and flatten() method
        
        if len(n_of_bins)==0:
            
            if feat == features[0]:
                print('Number of bins: Automatic')
            
            ax.hist(donors[feat])
            
        else:
            if feat == features[0]:
                print('Number of bins:', n_of_bins[0])
            ax.hist(donors[feat], bins = n_of_bins[0])
            
        ax.set_title(feat, y=-0.13)

    # Layout
    # Add a centered title to the figure:
    title = "Numeric Variables' Histograms"

    plt.suptitle(title)

    if savefig:
        # Save figure and include time stamp, the feature names and the method in the filename
        now_string = str(datetime.datetime.now())[0:19].replace(':', '-').replace(' ', '_')
        folder_path = os.path.join('figures')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        filename = now_string + '_' +  '-'.join(features) + '_histograms_' + str(n_of_bins[0]) + '_bins.png'
        plt.savefig(os.path.join(folder_path, filename), dpi=200)

In [ ]:
def remove_value(feature, value):
    """This function takes a feature name and a value and turns all elements of the respective column with this value
    into nans."""
    series = donors[feature].copy()
    series[series==value]=np.nan
    donors[feature] = series

In [ ]:
# Adapted from "lab04_data_visualization" (Data Mining)

def corr_matrix_visualisation(features, savefig, method):
    """This function takes a list of features and created a visualisation of their correlation matrix."""
    
    # Prepare figure
    fig = plt.figure(figsize=(25, 15))

    # Obtain correlation matrix. Round the values to 4 decimal cases. Use the DataFrame corr() and round() method.
    corr = np.round(donors[features].corr(method=method), decimals=4)

    # Build annotation matrix (values above |bound| will appear annotated in the plot)
    bound = 0
    mask_annot = np.absolute(corr.values) >= bound
    annot = np.where(mask_annot, corr.values, np.full(corr.shape,"")) # Try to understand what this np.where() does

    # Plot heatmap of the correlation matrix
    sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
                fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

    # Layout
    fig.subplots_adjust(top=0.95)
    fig.suptitle('Correlation Matrix' + ' (' + method + ')', fontsize=20)

    if savefig:
        # Save figure and include time stamp, the feature names and the method in the filename
        now_string = str(datetime.datetime.now())[0:19].replace(':', '-').replace(' ', '_')
        folder_path = os.path.join('figures')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        filename = now_string + '_' +  '-'.join(features) + '_' + method + '_corr_matrix.png'
        plt.savefig(os.path.join(folder_path, filename), dpi=200)

    plt.show()

In [ ]:
def uniques_nans_variance(feature):
    """This function takes the name of a feature and prints its unique values, its percentage of missing values and
    its standard deviation. Then (optionally) it returns the respective column so it is displayed as output."""
    
    print('Unique values:')
    unique = pd.unique(donors[feature])
    unique.sort()
    print(unique)
    print()
    
    print('Percentage of missing values:')
    print(round(len(donors[feature][donors[feature].isna()])/len(donors[feature]), 4), '%')
    print()
    
    print('Minimum value:', donors[feature].min(), '- Share:', \
          round(((len(donors[feature][donors[feature]==donors[feature].min()])) / len(donors[feature])), 6) * 100, '%')
    
    print('Maximum value:', donors[feature].max(), '- Share:', \
          round(((len(donors[feature][donors[feature]==donors[feature].max()])) / len(donors[feature])), 6) * 100, '%')
    print()
    
    try:
        print('Sample standard deviation:', round(donors[feature].std(), 4))
    except:
        print("Couldn't compute a variance")
        
    # return donors[feature]

In [ ]:
def df_nans_variance_min_max(features):
    """This function takes a list of features and computes their percentages of missing values, their sample
    standard deviations, their Minima and Maxima and the percentage of how much the minima and maxima are in
    this feature. Then it saves all this in a DataFrame which it returns."""
    nans = []
    st_devs = []
    mins = []
    min_shares = []
    maxs = []
    max_shares = []
    for feature in features:
        nans.append(round(len(donors[feature][donors[feature].isna()])/len(donors[feature]), 4))
        st_devs.append(round(donors[feature].std(), 4))
        mins.append(donors[feature].min())
        min_shares.append(round(((len(donors[feature][donors[feature]==donors[feature].min()])) \
                                 / len(donors[feature])),6) * 100)
        maxs.append(donors[feature].max())
        max_shares.append(round(((len(donors[feature][donors[feature]==donors[feature].max()])) \
                                 / len(donors[feature])),6) * 100)
    
    df = pd.DataFrame(data = {'Missing values [%]': nans, 'Sample standard deviation [Unit of feature]': st_devs, 'Minimum [Unit of f.]': mins,\
                              'Share of Minimum [%]:': min_shares, 'Maximum [Unit of f.]': maxs, 'Share of Maximum [%]': max_shares},\
                      index = features)
    return df

In [ ]:
def many_boxplots(features, savefig):
    
    # All Numeric Variables' Box Plots in one figure
    sns.set()

    # Prepare figure. Create individual axes where each box plot will be placed
    fig, axes = plt.subplots(2, ceil(len(features) / 2), figsize=(20, 11))

    # Plot data
    # Iterate across axes objects and associate each box plot (hint: use the ax argument):
    for ax, feat in zip(axes.flatten(), features): # Notice the zip() function and flatten() method
        sns.boxplot(x=donors[feat], ax=ax)

    # Layout
    # Add a centered title to the figure:
    title = "Numeric Variables' Box Plots"

    plt.suptitle(title)
    
    if savefig:
        # Save figure and include time stamp, the feature names and the method in the filename
        now_string = str(datetime.datetime.now())[0:19].replace(':', '-').replace(' ', '_')
        folder_path = os.path.join('figures')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        filename = now_string + '_' +  '-'.join(features) + '_boxplots.png'
        plt.savefig(os.path.join(folder_path, filename), dpi=200)
    
    plt.show()

In [ ]:
# Adapted from "lab04_data_visualization" (Data Mining)

def pairwise(features, flag_kdeplot):
    """This function takes a list of features and creates their pairwise scatterplots and each feature's histogram.
    If 'flag_kdeplot' = True, a Kernel Dnsity Etimation plot will be added."""
    
    if flag_kdeplot:
        print('Plotting with Kernel Density Estimation. This will increase computation time immensely.')
    else:
        print('Plotting without Kernel Density Estimation.')
    
    # Pairwise Relationship of Numerical Variables
    sns.set()

    # Setting pairplot
    g = sns.pairplot(donors[features], diag_kind="hist", corner=True)
    
    if flag_kdeplot:
        g.map_lower(sns.kdeplot, levels=4, color=".2")

    # Layout
    plt.subplots_adjust(top=0.95)
    plt.suptitle("Pairwise Relationship of Numerical Variables", fontsize=20)

    # Save figure and include time stamp, the feature names and the method in the filename
    now_string = str(datetime.datetime.now())[0:19].replace(':', '-').replace(' ', '_')
    folder_path = os.path.join('figures')
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    filename = now_string + '_' +  '-'.join(features) + '_pairwise_plots.png'
    plt.savefig(os.path.join(folder_path, filename), dpi=200)
    
    plt.show()

### Remove value 0 from columns 'AGE901', 'AGE902', 'AGE903', 'AGE904', 'AGE905', 'AGE906','AGE907'

In [ ]:
features = ['AGE901', 'AGE902', 'AGE903', 'AGE904', 'AGE905', 'AGE906','AGE907']

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(features, savefig, 100)

<span style="color:red">**None of these features have missing values. All of them have suspiciously many zeros. Assume: Missing values were saved as zeros. Remove zeros.**<span>

In [ ]:
# Replace zeros with nans
for feature in ['AGE901', 'AGE902', 'AGE903', 'AGE904', 'AGE905', 'AGE906','AGE907']:
    remove_value(feature, 0)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(features, savefig, 100)

### Remove value 0 from columns 'HHAGE1', 'AGEC6', 'HHAGE2', 'HHAGE3'?

In [ ]:
features = ['HHAGE1', 'AGEC6', 'HHAGE2', 'HHAGE3']

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(features, savefig, 250)

<span style="color:red">**None of these features have missing values. 'HHAGE1', 'AGEC6', 'HHAGE3' have many zeros in comparison to surrounding area in their histograms. Assume: Missing values were saved as zeros. Remove zeros from 'HHAGE1', 'AGEC6', 'HHAGE3'.**<span>

In [ ]:
# Replace zeros with nans
for feature in ['HHAGE1', 'AGEC6', 'HHAGE3']:
    remove_value(feature, 0)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(features, savefig, 250)

### Remove value 0 from columns 'HHN1', 'HHN2', 'HHN3', 'HHN4', 'HHN5', 'HHN6' ?

In [ ]:
features = ['HHN1', 'HHN2', 'HHN3', 'HHN4', 'HHN5', 'HHN6']

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(features, savefig, 180)

<span style="color:red">**Justifiable to remove zeros from HHN1, HHN2, HHN3 and HHN4. Same reasoning as above.**<span>

In [ ]:
# Replace zeros with nans
for feature in ['HHN1', 'HHN2', 'HHN3', 'HHN4']:
    remove_value(feature, 0)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(features, savefig, 180)

### Remove value 0 from columns 'HHP1', 'HHP2', 'HHN3', 'HHN4', 'HHD1', 'RHP3' ?

In [ ]:
features = [
    'HHP1', # ("Median Person Per Household")
    'HHP2', # ("Average Person Per Household")
    'HHN3', # ("Percent 3 or More Person Households")
    'HHN4', # ("Percent 4 or More Person Households")
    'HHD1', # ("Percent Households w/ Related Children")
    'RHP3' # ("Median Number of Persons per Housing Unit")
]

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(features, savefig, 200)

<span style="color:red">**Justifiable to remove zeros from HHP1, HHP2, HHD1 and RHP3. Same reasoning as above.**<span>
    
<span style="color:red">**Remark: RHP3 looks odd. Let's consider remving it. (Will be removed in the next step.)**<span>

In [ ]:
# Replace zeros with nans
for feature in ['HHP1', 'HHP2', 'HHD1', 'RHP3']:
    remove_value(feature, 0)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(features, savefig, 200)

### Remove value 0 from columns  'RP1', 'RP2', 'RP3', 'RP4', 'HV4' ?

In [ ]:
features = [
    'RP1', # ("Percent Renters Paying >= $500 per Month")
    'RP2', # ("Percent Renters Paying >= $400 per Month")
    'RP3', # ("Percent Renters Paying >= $300 per Month")
    'RP4', # ("Percent Renters Paying >= $200 per Month")
    'HV4', # ("Average Contract Rent in hundreds")
]

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(features, savefig, 200)

<span style="color:red">**Justifiable to remove zeros from RP3, RP4. Same reasoning as above.**<span>

In [ ]:
# Replace zeros with nans
for feature in ['RP3', 'RP4']:
    remove_value(feature, 0)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(features, savefig, 200)

### Remove value 0 from columns 'IC1', 'IC2', 'IC3', 'IC4', 'IC5' ?

In [ ]:
features = [
    'IC1', # ("Median Household Income in hundreds")
    'IC2', # ("Median Family Income in hundreds")
    'IC3', # ("Average Household Income in hundreds")
    'IC4', # ("Average Family Income in hundreds")
    'IC5', #("Per Capita Income")
]

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(features, savefig, 200)

<span style="color:red">**Justifiable to remove zeros from all of them. Same reasoning as above.**<span>

In [ ]:
# Replace zeros with nans
for feature in ['IC1','IC2','IC3','IC4','IC5']: 
    remove_value(feature, 0)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(features, savefig, 200)

### Remove value 0 from columns 'IC6', ... , 'IC23' ?

In [ ]:
households = [
    'IC6', # ("Percent Households w/ Income < $15,000")
    'IC7', # ("Percent Households w/ Income $15,000 - $24,999")
    'IC8', # ("Percent Households w/ Income $25,000 - $34,999")
    'IC9', # ("Percent Households w/ Income $35,000 - $49,999")
    'IC10', # ("Percent Households w/ Income $50,000 - $74,999")
    'IC11', # ("Percent Households w/ Income $75,000 - $99,999")
    'IC12', # ("Percent Households w/ Income $100,000 - $124,999")
    'IC13', # ("Percent Households w/ Income $125,000 - $149,999")
    'IC14', # ("Percent Households w/ Income >= $150,000")
]

families = [
    'IC15', # ("Percent Families w/ Income < $15,000")
    'IC16', # ("Percent Families w/ Income $15,000 - $24,999")
    'IC17', # ("Percent Families w/ Income $25,000 - 34,999")
    'IC18', # ("Percent Families w/ Income $35,000 - $49,999")
    'IC19', # ("Percent Families w/ Income $50,000 - $74,999")
    'IC20', # ("Percent Families w/ Income $75,000 - $99,999")
    'IC21', # ("Percent Families w/ Income $100,000 - $124,999")
    'IC22', # ("Percent Families w/ Income $125,000 - $149,999")
    'IC23' # ("Percent Families w/ Income >= $150,000")
]

features = households + families

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(households, savefig, 100)

In [ ]:
many_hists(families, savefig, 100)

<span style="color:red">**For the same reasoning as above, remove zeros from IC6, ..., IC10 and IC15, ... IC19. For the other ones there are probably a lot of "false zeros" as well, but the risk of deleting too many legitimate zeros is too high in my opinion.**<span>

In [ ]:
# Replace zeros with nans
for feature in [
    'IC6', # ("Percent Households w/ Income < $15,000")
    'IC7', # ("Percent Households w/ Income $15,000 - $24,999")
    'IC8', # ("Percent Households w/ Income $25,000 - $34,999")
    'IC9', # ("Percent Households w/ Income $35,000 - $49,999")
    'IC10', # ("Percent Households w/ Income $50,000 - $74,999")
    
    'IC15', # ("Percent Families w/ Income < $15,000")
    'IC16', # ("Percent Families w/ Income $15,000 - $24,999")
    'IC17', # ("Percent Families w/ Income $25,000 - 34,999")
    'IC18', # ("Percent Families w/ Income $35,000 - $49,999")
    'IC19', # ("Percent Families w/ Income $50,000 - $74,999")
]: 
    remove_value(feature, 0)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(households, savefig, 100)

In [ ]:
many_hists(families, savefig, 100)

### Remove value 0 from columns 'HHAS1', 'HHAS2', 'HHAS3', 'HHAS4' ?

In [ ]:
features = [
    'HHAS1', # ("Percent Households on Social Security")
    'HHAS2', # ("Percent Households on Public Assistance")
    'HHAS3', # ("Percent Households w/ Interest, Rental or Dividend Income")
    'HHAS4' # ("Percent Persons Below Poverty Level")
]

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(features, savefig, 200)

<span style="color:red">**For the same reasons as above, delete zeros from HHAS1, HHAS2 and HHAS3, but not HHAS4.**<span>

In [ ]:
# Replace zeros with nans
for feature in ['HHAS1','HHAS2','HHAS3',]: 
    remove_value(feature, 0)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(features, savefig, 200)

### Remove value 0 and 99 from columns 'LFC1', 'LFC2', 'LFC3', 'LFC4', 'LFC5', 'LFC6', 'LFC7', 'LFC8', 'LFC9', 'LFC10' 

In [ ]:
features = [
    'LFC1', # ("Percent Adults in Labor Force")
    'LFC2', # ("Percent Adult Males in Labor Force")
    'LFC3', # ("Percent Females in Labor Force")
    'LFC4', # ("Percent Adult Males Employed")
    'LFC5', # ("Percent Adult Females Employed")
    'LFC6', # ("Percent Mothers Employed Married and Single")
    'LFC7', # ("Percent 2 Parent Earner Families")
    'LFC8', # ("Percent Single Mother w/ Child in Labor Force")
    'LFC9', # ("Percent Single Father w/ Child in Labor Force")
    'LFC10' # ("Percent Families w/ Child w/ no Workers")
]

savefig = False

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
many_hists(features, savefig, 100)

<span style="color:red">**These 0 and 99 values are weird. Delete them. Also remove 50s from LFC9**<span>

In [ ]:
# Replace zeros with nans
for feature in features: 
    remove_value(feature, 0)

In [ ]:
# Replace 99s with nans
for feature in features: 
    remove_value(feature, 99)

In [ ]:
# Replace 50s with nans for LFC9
remove_value('LFC9', 50)

<span style="color:red">**After removing zeros:**<span>

In [ ]:
many_hists(features, savefig, 100)

### Remove value 0 for ACF4 and ACF5

In [ ]:
# Replace zeros with nans for ACF4 and ACF5
remove_value('AFC4', 0)
remove_value('AFC5', 0)

### Treat the cases where 'LASTDATE' suggests that donor is not lapsing or lapsed (In this dataset, all donors should be either lapsing or lapsed)

<span style="color:red">**We have to check this! I feel like I remove too many observations here.**<span>

Forum entry "Lapsed donors"

Entry by David Silva - Thursday, 3 December 2020, 9:43 AM
 	
"Hi Philipp,

Yes. A lapsed donor is a "snapshot" label so it depends on a point in time. The lapsed donors in this dataset are determined according to the date the last promotion (17NK) was emailed to each one of them and the date of their most recent gift. Inconsistent observations are the ones which have an interval between these two dates smaller than 13 months as according to the Lapsed donors description: "A previous donor who made their donation between 13-24 months ago"."

In [ ]:
# Only execute if flag=True
flag = False

In [ ]:
if flag:
    # Create a series that contains the dates from 'adate_2' substacted by 13 months
    adate_2_minus_13_months = donors.ADATE_2.map(lambda x: x - dateutil.relativedelta.relativedelta(months=13))
    adate_2_minus_13_months

In [ ]:
if flag:
    # Check if it worked properly
    (donors.ADATE_2 - adate_2_minus_13_months) / 13

In [ ]:
if flag:
    # Remove inconsistent values from donors
    # Inconsistent values are the ones where 'adate_2_minus_13_months' is before 'LASTDATE'
    # All observations that are not inconsistent are kept.
    donors_2 = donors[~(adate_2_minus_13_months < donors['LASTDATE'])]

    percentage_discarded = ((donors.shape[0] - donors_2.shape[0]) / donors.shape[0])*100
    print('Percentage of observation discarded due to inconsistency between their "lapsed" status and their LASTDATE value:')
    print(round(percentage_discarded, 2), '%')

In [ ]:
if flag:
    # Accept the changes
    donors = donors_2

## Discard features that add little additional information
**Approach: Identify which variables are highly correlated to each other (pandas-profiling is of great help here) and of these groups plot the covariances and their single and common distributions as well as compute their missing value percentages and their sample standard errors. With this information decide which ones to discard and which ones to keep.**

**Workflow: Paste features that are to be looked at into 'features' in the following cell. Then execute all cells of this step. Make decisions.**

In [ ]:
features = [
    'HC17', # ("Percent Housing Units w/ Public Water Source")
    'HC18' # ("Percent Housing Units w/ Well Water Source")
]

savefig = True

In [ ]:
if False:
    uniques_nans_variance(features[0])

In [ ]:
df_nans_variance_min_max(features)

In [ ]:
df_nans_variance_min_max(features)['Sample standard deviation [Unit of feature]']

In [ ]:
corr_matrix_visualisation(features, savefig, 'pearson')

In [ ]:
corr_matrix_visualisation(features, savefig, 'spearman')

In [ ]:
many_hists(features, savefig, 100)

In [ ]:
many_boxplots(features, savefig)

In [ ]:
t_before = datetime.datetime.now()

create_kdeplot = True

pairwise(features, create_kdeplot)

t_after = datetime.datetime.now()
print('Computation time = ', t_after - t_before)

In [ ]:
# Following direction of pandas profiling (html file in folder "pandas-profiling/2020-12-15 14/50/43.266760"),
# determine which features to drop right away:
to_drop = [
    
    # 'RFA_2R' # ("Recency code for RFA_2") 
    
    # Is a column with a constant value. No variance, no information. Drop it.
    
    'RFA_2R',
    
    #====================
    #====================
    
    # 'POP901', # ("Number of Persons")
    # 'POP902', # ("Number of Families")
    # 'POP903' # ("Number of Households")
    
    # 'POP902' and 'POP903' are both highly correlated with 'POP901'.
    
    # None of the three have missing values.
    
    # Keep 'POP901'. Drop the other two.
    
    'POP902', 
    'POP903',
    
    #====================
    #====================
    
    # 'ETH13', # ("Percent Mexican") 
    # 'ETH5', # ("Percent Hispanic")
    # 'LSC2' # ("Percent Spanish Speaking")
    
    # None have missing values.
    
    # Sample standard deviations:
    # ETH13    11.3335
    # ETH5     13.7861
    # LSC2     12.0437
    
    # -> Keep 'ETH5'
    
    'ETH13',
    'LSC2',
    
    #====================
    #====================
    
    # 'AGE901', # ("Median Age of Population")
    # 'AGE902', # ("Median Age of Adults 18 or Older")
    # 'AGE903', # ("Median Age of Adults 25 or Older"
    # 'AGE904', # ("Average Age of Population"). Carries slightly less information (in the variance sense) than 'AGE901': Sample standard deviations: 7.2612 years vs. 8.3356 years
    # 'AGE905', # ("Average Age of Adults >= 18")
    # 'AGE906' # ("Average Age of Adults >= 25")
    
    # The latter 5 are highly correlated with 'AGE901'.
    
    # All have the same amount of missing values.
    
    # Sample standard deviations:
    # AGE901    7.7426
    # AGE902    7.3261
    # AGE903    6.9989
    # AGE904    6.4949
    # AGE905    5.6426
    # AGE906    5.3142
    
    # -> Keep only 'AGE901'
    
    'AGE902', 
    'AGE903', 
    'AGE904', 
    'AGE905',
    'AGE906',
    
    #====================
    #====================
    
    # 'HHAGE1', # ("Percent Households w/ Person 65+"").
    # 'AGEC6', # ("Percent Adults Age 65-74")
    # 'HHAGE2', # ("Percent Households w/ Person 65+ Living Alone")
    # 'HHAGE3', # ("Percent Households Headed by an Elderly Person Age 65+")
    
    # 'AGEC6', 'HHAGE2', 'HHAGE3' are highly correlated with 'HHAGE1'.
    
    # These four features have a similar meaning.
    
    # None have missing values.
    
    # Sample standard deviations:
    # HHAGE1    13.0903
    # AGEC6      6.0038
    # HHAGE2     7.4415
    # HHAGE3    12.9639
    
    # -> 'HHAGE1' has the highest variance. Keep it. Drop the rest.
    
    'AGEC6',
    'HHAGE2',
    'HHAGE3',
    
    #====================
    #====================
    
    # 'HHN1', # ("Percent 1 Person Households")
    # 'HHN2', # ("Percent 2 Person Households")
    # 'HHN3', # ("Percent 3 or More Person Households")
    # 'HHN4', # ("Percent 4 or More Person Households")
    # 'HHN5', # ("Percent 5 or More Person Households")
    # 'HHN6' # ("Percent 6 Person Households")
    # 3, 4 and 5, 6 are highly correlated with each other and have similar meanings.
    # Sample standard deviations:
    # HHN3: 14.5385 %
    # HHN4: 11.0592 %
    # -> Keep HHN3
    # HHN5: 6.3828 %
    # HHN6: 3.793 %
    # -> Keep HHN5
    # -> Drop HHN4, HHN6:
    
    'HHN4',
    'HHN6',
    
    #====================
    #====================
    
    # 'MARR3', # ("Percent Widowed")
    # 'AGEC7' # ("Percent Adults Age >= 75")
    
    # When there are many old people, there are also many widowed people.
    
    # Both have no nan values.
    
    # -> Drop the one with smaller variance.
    
    # Sample standard deviation:
    # MARR3: 4.8877 %
    # AGEC7: 6.7237 %
    
    'MARR3',
    
    #====================
    #====================
    
    # 'HHP1', # ("Median Person Per Household")
    # 'HHP2', # ("Average Person Per Household")
    # 'HHN3', # ("Percent 3 or More Person Households")
    # 'HHN4', # ("Percent 4 or More Person Households")
    # 'HHD1', # ("Percent Households w/ Related Children")
    # 'RHP3' # ("Median Number of Persons per Housing Unit")
    
    # Sample standard deviations:
    # HHP1    50.0412 people
    # HHP2    49.9018 people
    # HHN3    14.5385 %
    # HHN4    11.0592 %
    # HHD1    13.0351 %
    # RHP3     2.5598 people
    
    # My personal opinion: 'HHD1' has a different meaning to the other 5. Let's keep it.
    
    # But the other 5 are pretty similar to each other.
    
    # None of them have missing values.
    
    # 'RHP3' and 'HHP2' have a lower variance than 'HHP1'. Let's drop them.
    # 'HHN4' has a lower variance than 'HHN3'. Let's drop it.
    
    # Out of 'HHP1' and 'HHN3', 'HHP1' is less correlated with 'HHD1' (which we decided to keep). 
    # This holds for both Pearson and Spearman correlation. So let's drop 'HHN3'
    
    'HHP2',
    'HHN4',
    'RHP3',
    'HHN3',
    
    #====================
    #====================
    
    # 'DW1',   # Percent Single Unit Structure
    # 'DW2',   # Percent Detached Single Unit Structure
    # 'DW3',   # Percent Duplex Structure
    # 'DW4',   # Percent Multi (2+) Unit Structures
    # 'DW5',   # Percent 3+ Unit Structures
    # 'DW6',   # Percent Housing Units in 5+ Unit Structure
    # 'DW7',   # Percent Group Quarters
    # 'DW8',   # Percent Institutional Group Quarters
    # 'DW9',   # Non-Institutional Group Quarters
    # 'HUPA2', # Percent Housing Units w/ >= 10 Units at the Address
    # 'HUPA6'  # Percent Renter Occupied, 5+ Units
    
    # Sample standard deviations:
    # DW1      24.9746 % (Drop this one and...)
    # DW2      26.3531 % (...keep this one)
    # DW3       5.3653 % (Pretty independant from the rest. Keep it.)
    # DW4      23.8544 % (This one is highly correlated with DW4, DW5, DW6 and HUPA2, HUPA6. Keep DW4 and drop them.)
    # DW5      22.6455 % (Drop)
    # DW6      20.4578 % (Drop)
    # DW7       5.9069 % (Highly correlated with DW8. Keep this one and drop DW8)
    # DW8       4.2548 % (Drop)
    # DW9       3.9695 % (I assume it's % as well) (Pretty independant from the rest. Keep it.)
    # HUPA2    17.1094 % (Drop)
    # HUPA6    17.8080 % (Drop)
    
    'DW1',
    'DW5',
    'DW6',
    'DW8',
    'HUPA2',
    'HUPA6'
    
    #====================
    #====================
    
    # 'RP1', # ("Percent Renters Paying >= $500 per Month")
    # 'RP2', # ("Percent Renters Paying >= $400 per Month")
    # 'RP3', # ("Percent Renters Paying >= $300 per Month")
    # 'RP4', # ("Percent Renters Paying >= $200 per Month")
    # 'HV4' # ("Average Contract Rent in hundreds")
    
    # These are all highly correlated.
    
    # Keeping only RP3 since it is amongs the ones with the highest variance (Note that HV4 can not be included in the
    # comparison because it is on a totally different scale) and since after the removal of zeros in the previous step
    # its distribution seems to be the most well behaved. Dropping the other ones.
    
    'RP1',
    'RP2',
    'RP4',
    'HV4',
    
    #====================
    #====================
    
    # 'IC1', # ("Median Household Income in hundreds")
    # 'IC2', # ("Median Family Income in hundreds")
    # 'IC3', # ("Average Household Income in hundreds")
    # 'IC4', # ("Average Family Income in hundreds")
    # 'IC5' # ("Per Capita Income")
    
    # IC2 has the highest variance out of the first four (Note that IC5 can not be included in the comparison because 
    # it is on a totally different scale) and it looks like the most well behaved distribution. Keep IC2 and drop the
    # other ones.
    
    'IC1',
    'IC3',
    'IC4',
    'IC5',
    
    #====================
    #====================
    
    # 'IC6', # ("Percent Households w/ Income < $15,000")
    # 'IC7', # ("Percent Households w/ Income $15,000 - $24,999")
    # 'IC8', # ("Percent Households w/ Income $25,000 - $34,999")
    # 'IC9', # ("Percent Households w/ Income $35,000 - $49,999")
    # 'IC10', # ("Percent Households w/ Income $50,000 - $74,999")
    # 'IC11', # ("Percent Households w/ Income $75,000 - $99,999")
    # 'IC12', # ("Percent Households w/ Income $100,000 - $124,999")
    # 'IC13', # ("Percent Households w/ Income $125,000 - $149,999")
    # 'IC14', # ("Percent Households w/ Income >= $150,000")
    # 'IC15', # ("Percent Families w/ Income < $15,000")
    # 'IC16', # ("Percent Families w/ Income $15,000 - $24,999")
    # 'IC17', # ("Percent Families w/ Income $25,000 - 34,999")
    # 'IC18', # ("Percent Families w/ Income $35,000 - $49,999")
    # 'IC19', # ("Percent Families w/ Income $50,000 - $74,999")
    # 'IC20', # ("Percent Families w/ Income $75,000 - $99,999")
    # 'IC21', # ("Percent Families w/ Income $100,000 - $124,999")
    # 'IC22', # ("Percent Families w/ Income $125,000 - $149,999")
    # 'IC23' # ("Percent Families w/ Income >= $150,000")
    
    # Naturally, IC6 is highly correlated with IC15 and so on. So having both sets of features would be quite 
    # redundant. 
    
    # Looking at the variance (see sample standard deviations table below), it could be said that IC6 / IC 15 and
    # IC10 / IC 19 give the most information. The first is an indicator for the amount of poor households, the latter
    # an indicator for the amount of "medium" households. Idea: Keep one of each: The one with the higher variance, so
    # keep IC6 and IC19 and drop the rest. IC6 and IC19 have rather well behaved distributions as well.
    
    # Sample standard deviations: 
    # IC   "Household"   IC  "Families"
    # 6    14.2979     15    12.1377
    # 7    7.6694      16    8.8146
    # 8    5.9860      17    7.2545
    # 9    7.1408      18    8.0242
    # 10   9.4480      19    10.2369
    # 11   5.8353      20    6.6662
    # 12   3.3463      21    3.9072
    # 13   1.9231      22    2.3217
    # 14   4.6407      23    5.5313
    
    'IC7',
    'IC8',
    'IC9',
    'IC10',
    'IC11',
    'IC12',
    'IC13',
    'IC14',
    'IC15',
    'IC16',
    'IC17',
    'IC18',
    'IC20',
    'IC21',
    'IC22',
    'IC23',
    
    #====================
    #====================
    
    # 'HHAGE1' # ("Percent Households w/ Person 65+") (Decided to keep this one, see above in this cell.)
    # 'HHAS1' # ("Percent Households on Social Security")
    # 'HHAS2' # ("Percent Households on Public Assistance")
    # 'HHAS3' # ("Percent Households w/ Interest, Rental or Dividend Income")
    # 'HHAS4' # ("Percent Persons Below Poverty Level")
    
    # HHAGE1 and HHAS1 are highly correlated. HHAS1 looks more well behaved. Keep HHAS1 Drop HHAGE1.
    
    'HHAGE1' # ("Percent Households w/ Person 65+")"
    
    # Confirming this decision by double checking with this combination of features:
    
    # 'HHAS1' # ("Percent Households on Social Security") 
    # 'AGEC6', # ("Percent Adults Age 65-74")
    # 'HHAGE1', # ("Percent Households w/ Person 65+"").
    # 'HHAGE2', # ("Percent Households w/ Person 65+ Living Alone")
    # 'HHAGE3', # ("Percent Households Headed by an Elderly Person Age 65+")
    
    # These are all highly correlated.
    
    # Sample standard deviations:
    # HHAS1     13.6223
    # AGEC6      5.9319
    # HHAGE1    12.9246
    # HHAGE2     7.4415
    # HHAGE3    12.8220
    
    # A look at the combination above confirms the decision to keep HHAS1 since it has the highest variance out of
    # these ones.
    
    #====================
    #====================
    
    # 'TPE1', # ("Percent Driving to Work Alone Car/Truck/Van")
    # 'TPE2', # ("Percent Carpooling Car/Truck/Van)")
    # 'TPE3', # ("Percent Using Public Transportation")
    # 'TPE4', # ("Percent Using Bus/Trolley")
    # 'TPE5', # ("Percent Using Railways")
    # 'TPE6', # ("Percent Using Taxi/Ferry")
    # 'TPE7', # ("Percent Using Motorcycles")
    # 'TPE8', # ("Percent Using Other Transportation")
    # 'TPE9' # ("Percent Working at Home/No Transportation")
    
    # Could think about dropping either TPE3 or TPE4 since they are correlated.
    
    # Also note: If used, it would be good to delete the zeros at least from TPE1 and TPE2
    
    #====================
    #====================
    
    # 'LFC1', # ("Percent Adults in Labor Force")
    # 'LFC2', # ("Percent Adult Males in Labor Force")
    # 'LFC3', # ("Percent Females in Labor Force")
    # 'LFC4', # ("Percent Adult Males Employed")
    # 'LFC5', # ("Percent Adult Females Employed")
    # 'LFC6', # ("Percent Mothers Employed Married and Single")
    # 'LFC7', # ("Percent 2 Parent Earner Families")
    # 'LFC8', # ("Percent Single Mother w/ Child in Labor Force")
    # 'LFC9', # ("Percent Single Father w/ Child in Labor Force")
    # 'LFC10' # ("Percent Families w/ Child w/ no Workers")
    
    # LFC1,2,3,4,5 are highly correlated. Keep LFC4 (highest variance). Drop the rest.
    
    'LFC1',
    'LFC2',
    'LFC3',
    'LFC4',
    'LFC5',
    
    #====================
    #====================

    # 'EIC1', # ("Percent Employed in Agriculture")
    # 'OCC9' # ("Percent Farmers")
    
    # Sample standard deviations:
    # EIC1    5.5708
    # OCC9    5.1068
    
    # -> Keep EIC1, drop OCC9
    
    'OCC9',
    
    #====================
    #====================
    
    # 'AFC1', # ("Percent Adults in Active Military Service")
    # 'AFC2', # ("Percent Males in Active Military Service")
    # 'AFC3', # ("Percent Females in Active Military Service")
    # 'AFC4', # ("Percent Adult Veterans Age 16+")
    # 'AFC5', # ("Percent Male Veterans Age 16+")
    # 'AFC6' # ("Percent Female Veterans Age 16+")
    
    # Sample standard deviations:
    # AFC1     3.1694
    # AFC2     4.9033
    # AFC3     1.0661
    # AFC4     5.2705
    # AFC5    10.4757
    # AFC6     1.6404
    
    # ACF4 and ACF5 are highly correlated. Drop AFC4 (lower variance).
    # Same for ACF1 and ACF2. Drop ACF1.
    
    'AFC1', # ("Percent Adults in Active Military Service")
    'AFC4', # ("Percent Adult Veterans Age 16+")
    
    #====================
    #====================
    
    # 'HC1',
    # 'HC2',
    # 'HC3',
    # 'HC4',
    # 'HC5',
    # 'HC6',
    # 'HC7', # ("Percent Owner Occupied Structures Built Since 1980")
    # 'HC8' # ("Percent Owner Occupied Structures Built Prior to 1860")
    
    # HC7, HC8 are highly negatively correlated.
    
    # Consider dropping HC3, ..., HC8. Not sure about HC1 and HC2.
    
    #====================
    #====================
    
    # 'HC17', # ("Percent Housing Units w/ Public Water Source")
    # 'HC18' # ("Percent Housing Units w/ Well Water Source")
    
    # Highly negatively correlated. Dopping one.
    
    'HC18',

    #====================
    #====================
    
    # 'CARDPROM', # ("Lifetime number of card promotions received to date.")
    # 'NUMPROM', # ("Lifetime number of promotions received to date")
    
    # Highly correlated. Drop one.
    
    'CARDPROM',
    
    #====================
    #====================
    
    # 'CARDGIFT', # ("Number of lifetime gifts to card promotions to date")
    # 'NGIFTALL', # ("Number of lifetime gifts to date")
    
    # Highly correlated. Drop one.
    
    'CARDGIFT'
    
]

print('Number of features to drop:', len(to_drop))
to_drop

### Fill missing values for all metric features

**For now, let's just fill all missing values with the features median**

In [ ]:
for feature in metric_features:
    without_nans = donors[feature].dropna()
    if feature in date_features:
        median = sorted(without_nans)[len(without_nans)//2]
    else:
        median = without_nans.median()
    donors[feature].fillna(value=median, inplace=True)

In [ ]:
# Get the total number of nan values for the metric features
donors[metric_features].isna().sum()[donors[metric_features].isna().sum()!=0].sum()

<span style="color:red">**No more nan values present in the metric features**<span>

In [ ]:
# Get the total number of nan values once more
donors.isna().sum()[donors.isna().sum()!=0].sum()

## Convert date features to numbers
**The idea here is to use the value of 'ADATE_2' (Date the 17NK promotion was mailed) of every obersvation as reference date.**

**The new values will be the number of days before the reference date. So for example DOB will be turned into the age in days relative to reference date.**

**Additionally - for better understandability - all transformed columns are renamed to ......_rel_in_days**

In [ ]:
# Make copies
adate_2 = donors['ADATE_2'].copy()
date_features_copy = date_features.copy()
metric_features_copy = metric_features.copy()

for feature in date_features:
    
    rel_time = adate_2 - donors[feature]
    rel_time_days = rel_time.dt.days
    donors[feature] = rel_time_days
    
    # Attach '_rel_in_days' to column name
    new_col_name = feature + '_rel_in_days'
    donors.rename(columns={feature : new_col_name}, inplace=True)
    
    # Update our lists 'metric_features' and 'date_features', Step 1
    date_features_copy[date_features_copy.index(feature)] = new_col_name
    metric_features_copy[metric_features_copy.index(feature)] = new_col_name
    
# Update our lists 'metric_features' and 'date_features', Step 2
date_features = date_features_copy
metric_features = metric_features_copy

In [ ]:
donors['ADATE_10_rel_in_days']

In [ ]:
donors['ADATE_2_rel_in_days']

In [ ]:
donors.head(3)

In [ ]:
date_features

In [ ]:
metric_features

## Descriptive statistics
<span style="color:red">**This part needs work</span>

In [ ]:
# Create some descriptive statistics
desc_stats = donors.describe(include="all")  # try with all and without all
# desc_stats = donors.describe()  # try with all and without all

In [ ]:
# View a certain one
feature = 'DOB_rel_in_days'
desc_stats[feature]

In [ ]:
donors[feature][~donors[feature].isna()].describe(include="all")

## Metric features that are in string format: Convert to float

In [ ]:
# Get all the data types of metric features, excluding the features defined in 'date_features'
dtypes_metric_without_dates = donors[list(set(metric_features) - set(date_features))].dtypes

In [ ]:
# Get the features from 'dtypes_metric_without_dates' that have type 'object'
dtypes_metric_without_dates[dtypes_metric_without_dates=='O']

<span style="color:red">**Due to the removal of features with high missing-values percentage, no metric features in string format are left**</span>

<span style="color:red">**Maybe the function below will be handy for something else**</span>

In [ ]:
# Define a function to convert numerical String features to float
def to_num(df, colname):
    """This function takes a dataframe and a column name and converts the column with this name to float"""    
    df[colname] = pd.to_numeric(df[colname],errors='coerce')

## Treat outliers

In [ ]:
iqr_multiplier = 1.5

# Get the quartiles. "numeric_only=False" so we also get quartiles for datetime columns.
q25 = donors[metric_features].quantile(0.25, axis=0, numeric_only=False)
q75 = donors[metric_features].quantile(0.75, axis=0, numeric_only=False)
iqr = (q75 - q25)

upper_lim = q75 + iqr_multiplier * iqr
lower_lim = q25 - iqr_multiplier * iqr

In [ ]:
q25

## Categorical features: One-hot encode

In [ ]:
donors[non_metric_features].head(5)

In [ ]:
pd.unique(donors[non_metric_features]['MDMAUD_R'])